## Zero Bond

In [ ]:
import pytest
import tensorflow as tf
import pandas as pd
import numpy as np
from scripts.utils.utils.utils import (
    ZeroBond,
    FinanceUtils,
    VisualizationHelper
)
from scripts.trainer.trainer import simulate

T = 8

T_dicts = {
    t: 0 for t in range(1, T)
}
results_t = {
    t: None for t in range(1, T)
}
# Steps per year
nsteps_per_year = 48
# Params for check
for t in range(1, T):
    nsteps = t * nsteps_per_year
    # Fixed params
    simple_params = {
        "T": t,
        "N_steps": nsteps,
        "dim": 1,
        "sigma": 0.01,
        "nsims": 1
    }
    data, _ = simulate(
        T = simple_params["T"], 
        N_steps = simple_params["N_steps"], 
        dim = simple_params["dim"],
        sigma = simple_params["sigma"], 
        nsims = simple_params["nsims"]
    )

    # Get data to process the ZeroBound
    delta_x = data.delta_x_0.values
    xt = data.X_0.values
    dt = data.dt.values
    t_unique = data.dt.unique()
    dict_C = {dt:FinanceUtils.C(dt, sigma_value = 0.01) for dt in t_unique}
    ct = data.apply(lambda x: dict_C[x['dt']], axis = 1)
    nt = ZeroBond.N_tensor(dt,xt,ct)
    # Convert to tensors
    xt = tf.convert_to_tensor(xt, dtype = tf.float64)
    delta_x = tf.convert_to_tensor(delta_x,dtype = tf.float64)
    dt = tf.convert_to_tensor(dt, dtype = tf.float64)
    ct = tf.Variable(np.float64(ct), name = 'ct', trainable=False)
    T = tf.Variable(np.float64(T), name = 'T', trainable=False)
    # Fix the batch size
    batch_size = int(xt.shape[0] / nsteps)
    # Real values
    v_real = ZeroBond.Z_tensor(xt, dt, T, ct)
    v_real_reshaped = tf.reshape(v_real,(batch_size,nsteps))
    n_tensor = ZeroBond.N_tensor(dt, xt, ct)
    # Derivative:
    xt = tf.Variable(xt, name = 'xn', trainable = True)
    dt = tf.Variable(dt, name = 'tn', trainable = False)
    ct = tf.Variable(np.float64(ct), name = 'ct', trainable=False)
    with tf.GradientTape() as tape:
        y = ZeroBond.Z_normalized(xt, dt, T, ct)
    grad_df = tape.gradient(y, {'xn':xt})
    grads = grad_df['xn']
    # Simulate - LGM step:
    grads_reshaped = tf.reshape(grads, (batch_size, nsteps))
    xt_reshaped = tf.reshape(xt, (batch_size, nsteps))
    delta_x_reshaped = tf.reshape(delta_x, (batch_size, nsteps))
    # Calculate the MVP
    v = np.ones((batch_size, nsteps)) * np.float64(v_real_reshaped[0, 0])
    for i in range(1, nsteps):
        v[:, i] = (v[:, i - 1] + grads_reshaped[:, i - 1] * delta_x_reshaped[:, i])
    # Calculate errors absolute
    v_real = np.array(tf.reshape(v_real_reshaped, -1))
    v_column = np.array(tf.reshape(v, -1)) * n_tensor.numpy()
    dt_list = np.array(dt)
    df_results = pd.DataFrame(zip(xt.numpy(), v_real, v_column, dt_list, n_tensor.numpy()), columns = ["xt", "v_real","v_est","dt", "n"])
    # Error
    df_results["absolute_error"] = (df_results.v_real - df_results.v_est).abs()
    # Store results
    results_t[t] = df_results.absolute_error.mean()

In [ ]:
VisualizationHelper.plot_multiple_series(df_results, x = "dt", values_column = ["v_real", "v_est"], xlabel="dt")
VisualizationHelper.plot_multiple_series(df_results, x = "xt", values_column = ["v_real", "v_est"], xlabel="xt")

#### Aggregated result

In [ ]:
results_t

## IRS

![IRS](images/irs/irs.png)
$$
IRS(t, x_t) = \mathcal{N}(x_t, t)A_{1,m}(x_t, t, T)\left(E^{1,m}[S_{i,m}(T_i, T)\vert F(t)]\right)
$$
Donde:
* $A_{i,m} = \tau\sum_{i}^m P(x_t, T_i, T_m)$, con $\tau$ constante para todos los pagos en nuestro caso.
* $\mathcal{N}(x_t, t) = \frac{1}{D(t)}exp(H(t)x_t + \frac{1}{2}H(t)^2\zeta(t))$
* $P(x_t, t, T) = N(x_t, t) E\left[ \frac{1}{N(x_T, T)} \vert F_t\right]$

En este caso el $E^{i,m}[S_{i,m}(T_i, T)\vert F(t)]$ es Martingala, entonces la fórmula queda como:
$$
IRS(t, x_t) = \mathcal{N}(x_t, t)A_{1,m}(x_t, t, T)\left(\frac{P(x_t, t, T_1) - P(x_t, t, T_m)}{A_{1,m}} - K\right)
$$

Reference: https://github.com/LechGrzelak/FinancialEngineering_IR_xVA/blob/main/Lecture%2005-Interest%20Rate%20Products/Lecture%2005-Interest%20Rate%20Products.pdf

$$
IRS(t_0) = (S_{m,n}(t_0) -K)A_{m,n}(t_0)
$$
with:
$$
S_{m,n}(t_0) = \frac{P(t_0, T_m) - P(t_0, T_n)}{A_{m,n}(t_0)}
$$
$t_0$ is the present time

Is $P(t, T_m)$ a Zero-Coupon Bond?

In [ ]:
import pytest
import tensorflow as tf
import pandas as pd
import numpy as np
from scripts.utils.utils.utils import (
    IRS,
    ZeroBond,
    FinanceUtils,
    VisualizationHelper
)
from scripts.trainer.trainer import simulate

T = 8
Tm = 10

T_dicts = {
    t: 0 for t in range(1, T)
}
results_t = {
    t: None for t in range(1, T)
}
# Steps per year
nsteps_per_year = 48
# Params for check
nsteps = T * nsteps_per_year
# Fixed params
simple_params = {
    "T": T,
    "N_steps": nsteps,
    "dim": 1,
    "sigma": 0.01,
    "nsims": 1
}
data, _ = simulate(
    T = simple_params["T"], 
    N_steps = simple_params["N_steps"], 
    dim = simple_params["dim"],
    sigma = simple_params["sigma"], 
    nsims = simple_params["nsims"]
)

In [ ]:
# Get data to process the ZeroBound
delta_x = data.delta_x_0.values
xt = data.X_0.values
dt = data.dt.values
t_unique = data.dt.unique()
dict_C = {dt:FinanceUtils.C(dt, sigma_value = 0.01) for dt in t_unique}
ct = data.apply(lambda x: dict_C[x['dt']], axis = 1)
nt = ZeroBond.N_tensor(dt,xt,ct)
# Convert to tensors
xt = tf.convert_to_tensor(xt, dtype = tf.float64)
delta_x = tf.convert_to_tensor(delta_x,dtype = tf.float64)
dt = tf.convert_to_tensor(dt, dtype = tf.float64)
ct = tf.convert_to_tensor(ct, dtype = tf.float64)
T = tf.convert_to_tensor(T, dtype = tf.float64)
Tm = tf.convert_to_tensor(Tm, dtype = tf.float64)
# Fix the batch size
batch_size = int(xt.shape[0] / nsteps)
# Real values
v_real = IRS.IRS_test(
    xn = xt, t = dt, Ti = T, Tm = Tm, ct = ct
)
v_real_reshaped = tf.reshape(v_real,(batch_size,nsteps))
n_tensor = ZeroBond.N_tensor(dt, xt, ct)
# Derivative:
xt = tf.Variable(xt, name = 'xn', trainable = True)
dt = tf.Variable(dt, name = 'tn', trainable = False)
ct = tf.Variable(np.float64(ct), name = 'ct', trainable=False)
T = tf.Variable(np.float64(T), name = 'T', trainable=False)
Tm = tf.Variable(np.float64(Tm), name = 'Tm', trainable=False)
with tf.GradientTape() as tape:
    y = IRS.IRS_test_normalized(
        xn = xt, t = dt, Ti = T, Tm = Tm, ct = ct
    )
grad_df = tape.gradient(y, {'xn':xt})
grads = grad_df['xn']
# Custom grads
h = 1e-14
x_p_h = IRS.IRS_test_normalized(
    xn = xt + h, t = dt, Ti = T, Tm = Tm, ct = ct
)
x_m_h = IRS.IRS_test_normalized(
    xn = xt - h, t = dt, Ti = T, Tm = Tm, ct = ct
)
grad_custom = (x_p_h - x_m_h)/(2 * h)
# Simulate - LGM step:
grads_reshaped = tf.reshape(grads, (batch_size, nsteps))
grad_custom_reshaped = tf.reshape(grad_custom, (batch_size, nsteps))
n_tensor_reshaped = tf.reshape(n_tensor, (batch_size, nsteps))
xt_reshaped = tf.reshape(xt, (batch_size, nsteps))
delta_x_reshaped = tf.reshape(delta_x, (batch_size, nsteps))
# Calculate the MVP
v = np.ones((batch_size, nsteps)) * np.float64(v_real_reshaped[0, 0])
v_custom = np.ones((batch_size, nsteps)) * np.float64(v_real_reshaped[0, 0])
for i in range(1, nsteps):
    v[:, i] = (v[:, i - 1] + grads_reshaped[:, i - 1] * delta_x_reshaped[:, i])
    v_custom[:, i] = (v_custom[:, i - 1] + grad_custom_reshaped[:, i - 1] * delta_x_reshaped[:, i])
# Calculate errors absolute
v_real = np.array(tf.reshape(v_real_reshaped, -1))
v_column = np.array(tf.reshape(v, -1)) * n_tensor.numpy()
v_custom = np.array(tf.reshape(v_custom, -1)) * n_tensor.numpy()
dt_list = np.array(dt)
df_results = pd.DataFrame(
    zip(xt.numpy(), v_real, v_column, v_custom, dt_list, n_tensor.numpy(), grads.numpy(), grad_custom.numpy()), 
    columns = ["xt", "v_real","v_est","v_est_custom","dt", "n", "grads", "grads_custom"])
# Error
df_results["absolute_error"] = (df_results.v_real - df_results.v_est).abs()
# Store results
results_t[int(T)] = df_results.absolute_error.mean()

In [ ]:
VisualizationHelper.plot_multiple_series(df_results, x = "dt", values_column = ["v_real", "v_est", "v_est_custom"], xlabel="dt")
VisualizationHelper.plot_multiple_series(df_results, x = "xt", values_column = ["v_real", "v_est", "v_est_custom"], xlabel="xt")

# Swaption

In [ ]:
import pytest
import tensorflow as tf
import pandas as pd
import numpy as np
from scripts.utils.utils.utils import (
    ZeroBond,
    Swaption,
    FinanceUtils,
    VisualizationHelper
)
from scripts.trainer.trainer import simulate

T = 8
Tm = 10

T_dicts = {
    t: 0 for t in range(1, T)
}
results_t = {
    t: None for t in range(1, T)
}
# Steps per year
nsteps_per_year = 48
# Params for check
nsteps = T * nsteps_per_year
# Fixed params
simple_params = {
    "T": T,
    "N_steps": nsteps,
    "dim": 1,
    "sigma": 0.01,
    "nsims": 1
}
data, _ = simulate(
    T = simple_params["T"], 
    N_steps = simple_params["N_steps"], 
    dim = simple_params["dim"],
    sigma = simple_params["sigma"], 
    nsims = simple_params["nsims"]
)

In [ ]:
# Get data to process the ZeroBound
delta_x = data.delta_x_0.values
xt = data.X_0.values
dt = data.dt.values
t_unique = data.dt.unique()
dict_C = {dt:FinanceUtils.C(dt, sigma_value = 0.01) for dt in t_unique}
ct = data.apply(lambda x: dict_C[x['dt']], axis = 1)
nt = ZeroBond.N_tensor(dt,xt,ct)
# Convert to tensors
xt = tf.convert_to_tensor(xt, dtype = tf.float64)
delta_x = tf.convert_to_tensor(delta_x,dtype = tf.float64)
dt = tf.convert_to_tensor(dt, dtype = tf.float64)
ct = tf.convert_to_tensor(ct, dtype = tf.float64)
T = tf.convert_to_tensor(T, dtype = tf.float64)
Tm = tf.convert_to_tensor(Tm, dtype = tf.float64)
# Fix the batch size
batch_size = int(xt.shape[0] / nsteps)
# Real values
v_real = Swaption.Swaption_test(
    xn = xt, t = dt, Ti = T, Tm = Tm, ct = ct
)
v_real_reshaped = tf.reshape(v_real,(batch_size,nsteps))
n_tensor = ZeroBond.N_tensor(dt, xt, ct)
# Derivative:
xt = tf.Variable(xt, name = 'xn', trainable = True)
dt = tf.Variable(dt, name = 'tn', trainable = False)
ct = tf.Variable(np.float64(ct), name = 'ct', trainable=False)
T = tf.Variable(np.float64(T), name = 'T', trainable=False)
Tm = tf.Variable(np.float64(Tm), name = 'Tm', trainable=False)
with tf.GradientTape() as tape:
    y = Swaption.Swaption_test_normalized(
        xn = xt, t = dt, Ti = T, Tm = Tm, ct = ct
    )
grad_df = tape.gradient(y, {'xn':xt})
grads = grad_df['xn']
# Simulate - LGM step:
grads_reshaped = tf.reshape(grads, (batch_size, nsteps))
xt_reshaped = tf.reshape(xt, (batch_size, nsteps))
delta_x_reshaped = tf.reshape(delta_x, (batch_size, nsteps))
# Calculate the MVP
v = np.ones((batch_size, nsteps)) * np.float64(v_real_reshaped[0, 0])
for i in range(1, nsteps):
    v[:, i] = (v[:, i - 1] + grads_reshaped[:, i - 1] * delta_x_reshaped[:, i])
# Calculate errors absolute
v_real = np.array(tf.reshape(v_real_reshaped, -1))
v_column = np.array(tf.reshape(v, -1)) * n_tensor.numpy()
dt_list = np.array(dt)
df_results = pd.DataFrame(
    zip(xt.numpy(), v_real, v_column, dt_list, n_tensor.numpy(), grads.numpy()), 
    columns = ["xt", "v_real","v_est","dt", "n", "grads"])
# Error
df_results["absolute_error"] = (df_results.v_real - df_results.v_est).abs()
# Store results
results_t[int(T)] = df_results.absolute_error.mean()

In [ ]:
VisualizationHelper.plot_multiple_series(df_results, x = "dt", values_column = ["v_real", "v_est"], xlabel="dt")
VisualizationHelper.plot_multiple_series(df_results, x = "xt", values_column = ["v_real", "v_est"], xlabel="xt")